# Fresh Products

In [14]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import sys

# Move two levels up (to the project root) and append the `src` folder
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Append src to sys.path
sys.path.append(src_path)

from src import query_engines, dataframe_visualizer

q = query_engines.QueryEngines()

# Current Picture

## Q1: Identify all kinds of segmentation

The first part is to make sure that we can identify the 5 different segments. These are Non top Partners, Top Partners, MFCs, Specialties and all Groceries partners together


In [15]:
q.prepare_query(
    'possible_segmentations.sql'
    ,params=None
    ,to_load_file='possible_segmentations'
    ,load_from_to_load_file='possible_segmentations'
)

possible_segmentations  = q.query_run_starburst()
possible_segmentations.sort_values(by='n_store_id', ascending=False)

,segment,n_store_id,n_store_address_id
2,Specialties,8348,15346
3,Non Top Partner,2723,6043
0,Top Partner,2279,10823
1,MFC,104,256


### Q1.1 How many of them are migrated

In [16]:
q.prepare_query(
    'possible_segmentations_m.sql'
    ,params=None
    ,to_load_file='possible_segmentations_m'
    ,load_from_to_load_file='possible_segmentations_m'
)

possible_segmentations_m  = q.query_run_starburst()
possible_segmentations_m.sort_values(by='n_store_id', ascending=False)

,segment,n_store_id,n_store_id_m,perc_m
3,Specialties,8348,480,0.06
1,Non Top Partner,2723,729,0.27
2,Top Partner,2279,1802,0.79
0,MFC,104,102,0.98


## Q2: Query for current picture questions

In [17]:
START_DATE = "'2024-07-01'"
END_DATE = "'2024-09-30'"

params = [
    {'name':'start_date', 'value':START_DATE},
    {'name':'end_date', 'value':END_DATE}
]

q.prepare_query(
    'current_picture.sql'
    ,params=params
    ,to_load_file='current_picture'
    ,load_from_to_load_file='current_picture'
)

current_picture  = q.query_run_starburst()
current_picture

,country,segment,all_orders,f_orders,all_customers,f_customers,all_products,f_products,fo_all_products,fo_f_products
0,CI,MFC,43670,17630,11203,5268,2616,267,2288.0,267.0
1,PT,MFC,169080,38683,51099,14510,4347,309,3977.0,309.0
2,PL,Specialties,77004,0,25130,0,1245,0,NaN,NaN
3,CI,Non Top Partner,116557,31699,26503,10080,12465,439,9835.0,439.0
4,PL,Top Partner,244353,155687,77491,54224,25917,988,24345.0,988.0
...,...,...,...,...,...,...,...,...,...,...
63,RS,Non Top Partner,8951,3019,4783,1946,4056,145,3427.0,145.0
64,IT,Specialties,23586,7539,11641,3920,12449,727,8494.0,727.0
65,PT,Specialties,6958,3168,3041,1303,2897,206,2433.0,206.0
66,KE,Top Partner,152072,55410,40832,19224,28979,1234,20862.0,1234.0


### Q2.1 % of customers that placed a FO in the last 3 months

In [18]:
tmp = current_picture.copy()

pivot_tmp = tmp.pivot(index='country', columns='segment', values=['f_customers','all_customers'])
pivot_tmp.columns = [f'{col[0]}_{col[1]}' for col in pivot_tmp.columns]
pivot_tmp.reset_index(inplace=True)
pivot_tmp.fillna(0, inplace=True)

pivot_tmp['perc_MFC'] = pivot_tmp['f_customers_MFC']/pivot_tmp['all_customers_MFC']
pivot_tmp['perc_Non Top Partner'] = pivot_tmp['f_customers_Non Top Partner']/pivot_tmp['all_customers_Non Top Partner']
pivot_tmp['perc_Top Partner'] = pivot_tmp['f_customers_Top Partner']/pivot_tmp['all_customers_Top Partner']
pivot_tmp['perc_Specialties'] = pivot_tmp['f_customers_Specialties']/pivot_tmp['all_customers_Specialties']

print(pivot_tmp['f_customers_MFC'].sum()/pivot_tmp['all_customers_MFC'].sum(),'all MFC')
print(pivot_tmp['f_customers_Non Top Partner'].sum()/pivot_tmp['all_customers_Non Top Partner'].sum(),'all Non Top Partner')
print(pivot_tmp['f_customers_Top Partner'].sum()/pivot_tmp['all_customers_Top Partner'].sum(),'all Top Partner')
print(pivot_tmp['f_customers_Specialties'].sum()/pivot_tmp['all_customers_Specialties'].sum(),'all Specialties')

pivot_tmp[['country','perc_MFC','perc_Non Top Partner','perc_Top Partner','perc_Specialties']]

0.35535505914154514 all MFC
0.29302197499586935 all Non Top Partner
0.5388240140077345 all Top Partner
0.22926559766380342 all Specialties


,country,perc_MFC,perc_Non Top Partner,perc_Top Partner,perc_Specialties
0,AM,NaN,0.047826,0.523312,NaN
1,BA,NaN,0.376448,0.432791,0.271255
2,BG,NaN,0.316374,0.702916,0.241087
3,CI,0.470231,0.380334,0.341721,0.394671
4,ES,0.383439,0.235789,0.577264,0.559850
5,GE,NaN,0.000000,0.445019,NaN
6,HR,NaN,0.034399,0.394567,0.244273
7,IT,0.409244,0.324836,0.533555,0.336741
8,KE,0.346414,0.359375,0.470807,NaN
9,KG,0.286914,NaN,0.348447,NaN


### Q2.2 % of Fresh Ordres from total ordres

In [19]:
tmp = current_picture.copy()

pivot_tmp = tmp.pivot(index='country', columns='segment', values=['f_orders','all_orders'])
pivot_tmp.columns = [f'{col[0]}_{col[1]}' for col in pivot_tmp.columns]
pivot_tmp.reset_index(inplace=True)
pivot_tmp.fillna(0, inplace=True)

pivot_tmp['perc_MFC'] = pivot_tmp['f_orders_MFC']/pivot_tmp['all_orders_MFC']
pivot_tmp['perc_Non Top Partner'] = pivot_tmp['f_orders_Non Top Partner']/pivot_tmp['all_orders_Non Top Partner']
pivot_tmp['perc_Top Partner'] = pivot_tmp['f_orders_Top Partner']/pivot_tmp['all_orders_Top Partner']
pivot_tmp['perc_Specialties'] = pivot_tmp['f_orders_Specialties']/pivot_tmp['all_orders_Specialties']

print(pivot_tmp['f_orders_MFC'].sum()/pivot_tmp['all_orders_MFC'].sum(),'all MFC')
print(pivot_tmp['f_orders_Non Top Partner'].sum()/pivot_tmp['all_orders_Non Top Partner'].sum(),'all Non Top Partner')
print(pivot_tmp['f_orders_Top Partner'].sum()/pivot_tmp['all_orders_Top Partner'].sum(),'all Top Partner')
print(pivot_tmp['f_orders_Specialties'].sum()/pivot_tmp['all_orders_Specialties'].sum(),'all Specialties')

pivot_tmp[['country','perc_MFC','perc_Non Top Partner','perc_Top Partner','perc_Specialties']]

0.2605517360882309 all MFC
0.2572791840397474 all Non Top Partner
0.44692135646242154 all Top Partner
0.20063520162445006 all Specialties


,country,perc_MFC,perc_Non Top Partner,perc_Top Partner,perc_Specialties
0,AM,NaN,0.022609,0.420517,NaN
1,BA,NaN,0.350348,0.364140,0.242144
2,BG,NaN,0.303056,0.590378,0.233033
3,CI,0.403710,0.271961,0.258892,0.384458
4,ES,0.284500,0.240620,0.486738,0.539542
5,GE,NaN,0.000000,0.366352,NaN
6,HR,NaN,0.030785,0.354919,0.249721
7,IT,0.326581,0.258775,0.436163,0.319639
8,KE,0.233708,0.321429,0.364367,NaN
9,KG,0.227975,NaN,0.295238,NaN


### Q2.3 In Fresh Ordres what is the % of Fresh Products

In [20]:
tmp = current_picture.copy()

pivot_tmp = tmp.pivot(index='country', columns='segment', values=['fo_all_products','fo_f_products'])
pivot_tmp.columns = [f'{col[0]}_{col[1]}' for col in pivot_tmp.columns]
pivot_tmp.reset_index(inplace=True)
pivot_tmp.fillna(0, inplace=True)

pivot_tmp['perc_MFC'] = pivot_tmp['fo_f_products_MFC']/pivot_tmp['fo_all_products_MFC']
pivot_tmp['perc_Non Top Partner'] = pivot_tmp['fo_f_products_Non Top Partner']/pivot_tmp['fo_all_products_Non Top Partner']
pivot_tmp['perc_Top Partner'] = pivot_tmp['fo_f_products_Top Partner']/pivot_tmp['fo_all_products_Top Partner']
pivot_tmp['perc_Specialties'] = pivot_tmp['fo_f_products_Specialties']/pivot_tmp['fo_all_products_Specialties']

print(pivot_tmp['fo_f_products_MFC'].sum()/pivot_tmp['fo_all_products_MFC'].sum(),'all MFC')
print(pivot_tmp['fo_f_products_Non Top Partner'].sum()/pivot_tmp['fo_all_products_Non Top Partner'].sum(),'all Non Top Partner')
print(pivot_tmp['fo_f_products_Top Partner'].sum()/pivot_tmp['fo_all_products_Top Partner'].sum(),'all Top Partner')
print(pivot_tmp['fo_f_products_Specialties'].sum()/pivot_tmp['fo_all_products_Specialties'].sum(),'all Specialties')

pivot_tmp[['country','perc_MFC','perc_Non Top Partner','perc_Top Partner','perc_Specialties']]

0.05690307849460212 all MFC
0.0655861197511664 all Non Top Partner
0.04850267950036324 all Top Partner
0.14925196850393702 all Specialties


,country,perc_MFC,perc_Non Top Partner,perc_Top Partner,perc_Specialties
0,AM,NaN,0.067039,0.055505,NaN
1,BA,NaN,0.097187,0.032302,0.791667
2,BG,NaN,0.129870,0.043162,0.199485
3,CI,0.116696,0.044637,0.046206,0.332390
4,ES,0.035389,0.043102,0.045199,0.068256
5,GE,NaN,NaN,0.044321,NaN
6,HR,NaN,0.110899,0.033395,0.314614
7,IT,0.047270,0.055629,0.048510,0.085590
8,KE,0.093330,0.244898,0.059151,NaN
9,KG,0.040123,NaN,0.048174,NaN


## Q3: Fresh Sessions CVR%

In [21]:
START_DATE = "'2024-07-01'"
END_DATE = "'2024-09-30'"

params = [
    {'name':'start_date', 'value':START_DATE},
    {'name':'end_date', 'value':END_DATE}
]

q.prepare_query(
    'sessions.sql'
    ,params=params
    ,to_load_file='sessions'
    ,load_from_to_load_file='sessions'
)

sessions  = q.query_run_starburst()

tmp = sessions.copy()
pivot_tmp = tmp.pivot(index='country', columns='segment',values=['f_sessions','o_sessions'])
pivot_tmp.columns = [f'{col[0]}_{col[1]}' for col in pivot_tmp.columns]
pivot_tmp.reset_index(inplace=True)
pivot_tmp.fillna(0, inplace=True)

pivot_tmp['cvr_MFC'] = pivot_tmp['o_sessions_MFC']/pivot_tmp['f_sessions_MFC']
pivot_tmp['cvr_Non Top Partner'] = pivot_tmp['o_sessions_Non Top Partner']/pivot_tmp['f_sessions_Non Top Partner']
pivot_tmp['cvr_Top Partner'] = pivot_tmp['o_sessions_Top Partner']/pivot_tmp['f_sessions_Top Partner']
pivot_tmp['cvr_Specialties'] = pivot_tmp['o_sessions_Specialties']/pivot_tmp['f_sessions_Specialties']

print(pivot_tmp['o_sessions_MFC'].sum()/pivot_tmp['f_sessions_MFC'].sum(),'all MFC')
print(pivot_tmp['o_sessions_Non Top Partner'].sum()/pivot_tmp['f_sessions_Non Top Partner'].sum(),'all Non Top Partner')
print(pivot_tmp['o_sessions_Top Partner'].sum()/pivot_tmp['f_sessions_Top Partner'].sum(),'all Top Partner')
print(pivot_tmp['o_sessions_Specialties'].sum()/pivot_tmp['f_sessions_Specialties'].sum(),'all Specialties')
pivot_tmp[['country','cvr_MFC','cvr_Non Top Partner','cvr_Top Partner','cvr_Specialties']]

0.26376796621501336 all MFC
0.16059320450470582 all Non Top Partner
0.3311238329606336 all Top Partner
0.15225156456771063 all Specialties


,country,cvr_MFC,cvr_Non Top Partner,cvr_Top Partner,cvr_Specialties
0,0,0.177033,0.147059,0.256917,0.250000
1,AL,1.000000,NaN,0.000000,0.000000
2,AM,NaN,0.126339,0.341432,NaN
3,BA,0.000000,0.054589,0.179241,0.064070
4,BG,NaN,0.084583,0.286093,0.132749
5,CI,0.249950,0.318334,0.156676,0.111872
6,ES,0.293456,0.151464,0.415414,0.098324
7,GE,NaN,0.014528,0.197234,NaN
8,HR,0.166667,0.129308,0.229570,0.152059
9,IT,0.274737,0.091473,0.259443,0.115505


# Value of nudging users to Fresh

## Q4: Query for nuding users to Fresh questions

In [22]:
START_DATE = "'2024-07-01'"
END_DATE = "'2024-08-31'"

params = [
    {'name':'start_date', 'value':START_DATE},
    {'name':'end_date', 'value':END_DATE}
]

q.prepare_query(
    'nudging.sql'
    ,params=params
    ,to_load_file='nudging'
    ,load_from_to_load_file='nudging'
)

nudging  = q.query_run_starburst()
print('Data Considerations:')
print('1st July to 31st August')
print('only delivered orders')
print('only parent orders')
nudging

Data Considerations:
1st July to 31st August
only delivered orders
only parent orders


,country,segment,f_orders,nf_orders,f_value,nf_value,f_orders_ret,nf_orders_ret,f_orders_pna,nf_orders_pna,f_products,nf_products
0,ES,Top Partner,390783,414407,1.365439e+07,9.868862e+06,327785,324489,192623,126914,5432248,3166518
1,NG,MFC,9628,22780,6.226931e+04,1.044350e+05,8491,18242,503,736,47587,69415
2,KG,Top Partner,5126,11886,6.140321e+04,9.677291e+04,3727,7811,280,329,40501,45414
3,ES,Specialties,7267,5973,3.106255e+05,1.733494e+05,6521,4992,3626,1819,73778,30982
4,PT,Specialties,1960,2300,6.746234e+04,5.164637e+04,1771,1823,1059,556,19363,10156
...,...,...,...,...,...,...,...,...,...,...,...,...
63,PT,Non Top Partner,53,3641,1.013960e+03,4.382986e+04,39,2288,30,636,479,17485
64,BG,Top Partner,29778,20474,9.085757e+05,4.483035e+05,22863,15211,11999,5160,372926,142482
65,UA,Non Top Partner,550,4575,1.825037e+04,1.117330e+05,458,3708,25,41,3034,10579
66,AM,Top Partner,7563,9717,2.092273e+05,1.756128e+05,6404,7619,2948,1639,103438,70316


### Q4.1 Retention 1st degree

In [23]:
tmp = nudging.copy()
pivot_tmp = tmp.pivot(index='country',columns='segment',values=['f_orders','nf_orders','f_orders_ret','nf_orders_ret'])
pivot_tmp.columns = [f'{col[0]}_{col[1]}' for col in pivot_tmp.columns]
pivot_tmp.reset_index(inplace=True)
pivot_tmp.fillna(0, inplace=True)

pivot_tmp['f_perc_MFC'] = pivot_tmp['f_orders_ret_MFC']/pivot_tmp['f_orders_MFC']
pivot_tmp['nf_perc_MFC'] = pivot_tmp['nf_orders_ret_MFC']/pivot_tmp['nf_orders_MFC']
pivot_tmp['f_perc_Non Top Partner'] = pivot_tmp['f_orders_ret_Non Top Partner']/pivot_tmp['f_orders_Non Top Partner']
pivot_tmp['nf_perc_Non Top Partner'] = pivot_tmp['nf_orders_ret_Non Top Partner']/pivot_tmp['nf_orders_Non Top Partner']
pivot_tmp['f_perc_Top Partner'] = pivot_tmp['f_orders_ret_Top Partner']/pivot_tmp['f_orders_Top Partner']
pivot_tmp['nf_perc_Top Partner'] = pivot_tmp['nf_orders_ret_Top Partner']/pivot_tmp['nf_orders_Top Partner']
pivot_tmp['f_perc_Specialties'] = pivot_tmp['f_orders_ret_Specialties']/pivot_tmp['f_orders_Specialties']
pivot_tmp['nf_perc_Specialties'] = pivot_tmp['nf_orders_ret_Specialties']/pivot_tmp['nf_orders_Specialties']

pivot_tmp[['country','f_perc_MFC', 'nf_perc_MFC', 'f_perc_Non Top Partner', 'nf_perc_Non Top Partner', 'f_perc_Top Partner', 'nf_perc_Top Partner', 'f_perc_Specialties', 'nf_perc_Specialties']]

,country,f_perc_MFC,nf_perc_MFC,f_perc_Non Top Partner,nf_perc_Non Top Partner,f_perc_Top Partner,nf_perc_Top Partner,f_perc_Specialties,nf_perc_Specialties
0,AM,NaN,NaN,0.823529,0.829726,0.846754,0.784090,NaN,NaN
1,BA,NaN,NaN,0.652695,0.582456,0.665946,0.591718,0.692308,0.379518
2,BG,NaN,NaN,0.803456,0.714573,0.767782,0.742942,0.839262,0.723281
3,CI,0.889485,0.822878,0.876521,0.800019,0.847170,0.805769,0.855243,0.793233
4,ES,0.807753,0.750769,0.856781,0.724880,0.838790,0.783020,0.897344,0.835761
5,GE,NaN,NaN,NaN,0.680115,0.851914,0.783513,NaN,NaN
6,HR,NaN,NaN,0.684713,0.526879,0.698681,0.613581,0.740434,0.622670
7,IT,0.732795,0.680442,0.696653,0.669251,0.691496,0.649395,0.768939,0.659839
8,KE,0.911838,0.854321,1.000000,0.416667,0.837179,0.780721,NaN,NaN
9,KG,0.823688,0.733536,NaN,NaN,0.727078,0.657160,NaN,NaN


### Q4.2 PNA

In [24]:
tmp = nudging.copy()
pivot_tmp = tmp.pivot(index='country',columns='segment',values=['f_orders_pna','nf_orders_pna','f_orders','nf_orders'])
pivot_tmp.columns = [f'{col[0]}_{col[1]}' for col in pivot_tmp.columns]
pivot_tmp.reset_index(inplace=True)
pivot_tmp.fillna(0, inplace=True)

pivot_tmp['f_perc_MFC_pna'] = pivot_tmp['f_orders_pna_MFC']/pivot_tmp['f_orders_MFC']
pivot_tmp['nf_perc_MFC_pna'] = pivot_tmp['nf_orders_pna_MFC']/pivot_tmp['nf_orders_MFC']
pivot_tmp['f_perc_Non Top Partner_pna'] = pivot_tmp['f_orders_pna_Non Top Partner']/pivot_tmp['f_orders_Non Top Partner']
pivot_tmp['nf_perc_Non Top Partner_pna'] = pivot_tmp['nf_orders_pna_Non Top Partner']/pivot_tmp['nf_orders_Non Top Partner']
pivot_tmp['f_perc_Top Partner_pna'] = pivot_tmp['f_orders_pna_Top Partner']/pivot_tmp['f_orders_Top Partner']
pivot_tmp['nf_perc_Top Partner_pna'] = pivot_tmp['nf_orders_pna_Top Partner']/pivot_tmp['nf_orders_Top Partner']
pivot_tmp['f_perc_Specialties_pna'] = pivot_tmp['f_orders_pna_Specialties']/pivot_tmp['f_orders_Specialties']
pivot_tmp['nf_perc_Specialties_pna'] = pivot_tmp['nf_orders_pna_Specialties']/pivot_tmp['nf_orders_Specialties']

pivot_tmp[['country','f_perc_MFC_pna', 'nf_perc_MFC_pna', 'f_perc_Non Top Partner_pna', 'nf_perc_Non Top Partner_pna', 'f_perc_Top Partner_pna', 'nf_perc_Top Partner_pna', 'f_perc_Specialties_pna', 'nf_perc_Specialties_pna']]

,country,f_perc_MFC_pna,nf_perc_MFC_pna,f_perc_Non Top Partner_pna,nf_perc_Non Top Partner_pna,f_perc_Top Partner_pna,nf_perc_Top Partner_pna,f_perc_Specialties_pna,nf_perc_Specialties_pna
0,AM,NaN,NaN,0.470588,0.240981,0.389792,0.168673,NaN,NaN
1,BA,NaN,NaN,0.125749,0.070175,0.291892,0.172050,0.000000,0.078313
2,BG,NaN,NaN,0.237581,0.104523,0.402948,0.252027,0.192358,0.153687
3,CI,0.072622,0.030682,0.236607,0.145710,0.146226,0.096795,0.163537,0.109962
4,ES,0.047232,0.019310,0.495972,0.239752,0.492916,0.306254,0.498968,0.304537
5,GE,NaN,NaN,NaN,0.031700,0.433074,0.251654,NaN,NaN
6,HR,NaN,NaN,0.092357,0.098208,0.332934,0.211092,0.111936,0.072571
7,IT,0.054411,0.031243,0.648013,0.418974,0.623787,0.400724,0.496313,0.174906
8,KE,0.060736,0.015557,0.000000,0.250000,0.143059,0.118403,NaN,NaN
9,KG,0.082997,0.039681,NaN,NaN,0.054623,0.027680,NaN,NaN


### Q4.3 AOV

In [25]:
tmp = nudging.copy()
pivot_tmp = tmp.pivot(index='country',columns='segment',values=['f_value','nf_value','f_orders','nf_orders'])
pivot_tmp.columns = [f'{col[0]}_{col[1]}' for col in pivot_tmp.columns]
pivot_tmp.reset_index(inplace=True)
pivot_tmp.fillna(0, inplace=True)

pivot_tmp['f_aov_MFC'] = pivot_tmp['f_value_MFC']/pivot_tmp['f_orders_MFC']
pivot_tmp['nf_aov_MFC'] = pivot_tmp['nf_value_MFC']/pivot_tmp['nf_orders_MFC']
pivot_tmp['f_aov_Non Top Partner'] = pivot_tmp['f_value_Non Top Partner']/pivot_tmp['f_orders_Non Top Partner']
pivot_tmp['nf_aov_Non Top Partner'] = pivot_tmp['nf_value_Non Top Partner']/pivot_tmp['nf_orders_Non Top Partner']
pivot_tmp['f_aov_Top Partner'] = pivot_tmp['f_value_Top Partner']/pivot_tmp['f_orders_Top Partner']
pivot_tmp['nf_aov_Top Partner'] = pivot_tmp['nf_value_Top Partner']/pivot_tmp['nf_orders_Top Partner']
pivot_tmp['f_aov_Specialties'] = pivot_tmp['f_value_Specialties']/pivot_tmp['f_orders_Specialties']
pivot_tmp['nf_aov_Specialties'] = pivot_tmp['nf_value_Specialties']/pivot_tmp['nf_orders_Specialties']

pivot_tmp[['country','f_aov_MFC', 'nf_aov_MFC', 'f_aov_Non Top Partner', 'nf_aov_Non Top Partner', 'f_aov_Top Partner', 'nf_aov_Top Partner', 'f_aov_Specialties', 'nf_aov_Specialties']]

,country,f_aov_MFC,nf_aov_MFC,f_aov_Non Top Partner,nf_aov_Non Top Partner,f_aov_Top Partner,nf_aov_Top Partner,f_aov_Specialties,nf_aov_Specialties
0,AM,NaN,NaN,18.315184,14.997517,27.664594,18.072744,NaN,NaN
1,BA,NaN,NaN,23.632346,12.565645,27.191216,16.385100,19.272317,11.084614
2,BG,NaN,NaN,22.026659,17.050452,30.511642,21.896234,27.872946,21.534985
3,CI,12.594565,9.725850,19.791684,13.219637,23.184572,14.106008,19.362873,18.834588
4,ES,31.786228,22.010283,35.741485,20.412222,34.941117,23.814420,42.744665,29.022161
5,GE,NaN,NaN,NaN,13.474396,21.250970,14.327067,NaN,NaN
6,HR,NaN,NaN,17.952771,12.496155,24.702101,16.225460,22.852847,18.159476
7,IT,33.903534,21.267659,34.650282,23.201932,35.274653,22.927988,42.694163,30.023006
8,KE,12.132197,9.691814,9.258275,7.425025,17.346507,12.569704,NaN,NaN
9,KG,17.038845,11.201561,NaN,NaN,11.978778,8.141756,NaN,NaN


### Q4.4 UIPO

In [26]:
tmp = nudging.copy()
pivot_tmp = tmp.pivot(index='country',columns='segment',values=['f_products','nf_products','f_orders','nf_orders'])
pivot_tmp.columns = [f'{col[0]}_{col[1]}' for col in pivot_tmp.columns]
pivot_tmp.reset_index(inplace=True)
pivot_tmp.fillna(0, inplace=True)

pivot_tmp['f_uipo_MFC'] = pivot_tmp['f_products_MFC']/pivot_tmp['f_orders_MFC']
pivot_tmp['nf_uipo_MFC'] = pivot_tmp['nf_products_MFC']/pivot_tmp['nf_orders_MFC']
pivot_tmp['f_uipo_Non Top Partner'] = pivot_tmp['f_products_Non Top Partner']/pivot_tmp['f_orders_Non Top Partner']
pivot_tmp['nf_uipo_Non Top Partner'] = pivot_tmp['nf_products_Non Top Partner']/pivot_tmp['nf_orders_Non Top Partner']
pivot_tmp['f_uipo_Top Partner'] = pivot_tmp['f_products_Top Partner']/pivot_tmp['f_orders_Top Partner']
pivot_tmp['nf_uipo_Top Partner'] = pivot_tmp['nf_products_Top Partner']/pivot_tmp['nf_orders_Top Partner']
pivot_tmp['f_uipo_Specialties'] = pivot_tmp['f_products_Specialties']/pivot_tmp['f_orders_Specialties']
pivot_tmp['nf_uipo_Specialties'] = pivot_tmp['nf_products_Specialties']/pivot_tmp['nf_orders_Specialties']

pivot_tmp[['country','f_uipo_MFC', 'nf_uipo_MFC', 'f_uipo_Non Top Partner', 'nf_uipo_Non Top Partner', 'f_uipo_Top Partner', 'nf_uipo_Top Partner', 'f_uipo_Specialties', 'nf_uipo_Specialties']]

,country,f_uipo_MFC,nf_uipo_MFC,f_uipo_Non Top Partner,nf_uipo_Non Top Partner,f_uipo_Top Partner,nf_uipo_Top Partner,f_uipo_Specialties,nf_uipo_Specialties
0,AM,NaN,NaN,9.470588,6.825397,13.676848,7.236390,NaN,NaN
1,BA,NaN,NaN,10.922156,3.807018,13.012973,6.713458,7.230769,2.162651
2,BG,NaN,NaN,9.049676,5.299497,12.523541,6.959168,7.208169,4.239229
3,CI,5.919114,2.581710,6.711745,3.551086,6.993396,3.731731,3.597027,1.811090
4,ES,7.827102,3.875508,13.231906,5.687694,13.900932,7.641082,10.152470,5.187008
5,GE,NaN,NaN,NaN,3.368876,9.940147,5.054587,NaN,NaN
6,HR,NaN,NaN,5.378981,3.246609,9.606563,4.916385,4.213307,2.565806
7,IT,8.451123,4.024475,12.848849,6.745293,16.674979,8.406395,10.167374,3.862520
8,KE,5.875062,2.346832,3.500000,3.166667,7.697308,4.282521,NaN,NaN
9,KG,7.420592,3.646677,NaN,NaN,7.901092,3.820798,NaN,NaN
